In [1]:
from utils import *
import os
import cv2
import numpy as np
import random as ra

In [2]:
#for color binarization:
# def extract_pallet_pixels(DS_path):
    
# def extract_background_pixels(DS_path):

def include_border(x1,y1,x2,y2, border):
    x1+=border
    x2+=border
    y1+=border
    y2+=border
    return x1,y1,x2,y2

def correct_rect_ratio(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    center=((x1+x2)/2,(y1+y2)/2)
    h=w/5 #ratio 1:5
    y1=int(center[1]-h/2)
    y2=int(center[1]+h/2)
    return x1,y1,x2,y2

def add_margin(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    center=((x1+x2)/2,(y1+y2)/2)
    w_margin=0.2*w/2
    h_margin=0.2*h/2
    x1-=w_margin
    x2+=w_margin
    y1-=h_margin
    y2+=h_margin
    return int(x1),int(y1),int(x2),int(y2)

def update_optimal_dimensions(x1,y1,x2,y2):
    w,h=(abs(x1-x2), abs(y1-y2))
    global w_max,h_max,w_min,h_min
    w_max=max(w,w_max)
    h_max=max(h,h_max)
    w_min=min(w,w_min)
    h_min=min(h,h_min) 
    
    
def extract_pallet_rectangles_from_image(regions, border, image_bordered, filename, scene_name):
    for j in range(len(regions)):
        (x1,y1),(x2,y2)=regions[j]
        x1,y1,x2,y2=include_border(x1,y1,x2,y2,border)
        x1,y1,x2,y2=correct_rect_ratio(x1,y1,x2,y2)
        update_optimal_dimensions(x1,y1,x2,y2)
        x1,y1,x2,y2=add_margin(x1,y1,x2,y2)
        cropped_m = image_bordered[y1:y2, x1:x2] #fixme calculaing max may be required
        cropped_m = cv2.resize(cropped_m, (120,24))
        cropped_filename=filename.replace(".jpg", "_"+str(j)+".jpg")
        pallet_rectangles_margin_path = DS_path+"processed_scenes/pallet_rectangles_margin"
        os.chdir(pallet_rectangles_margin_path) 
        try:
            os.mkdir(scene_name)
        except FileExistsError:
            None
        os.chdir(scene_name) 
        cv2.imwrite(cropped_filename, cropped_m)
        
def extract_background_rectangles_from_image(regions, image, filename, scene_name):
    if regions:
        pallets_y1=99999
        pallets_y2=0
        sample_pallet_w=0
        y1s=[]
        y2s=[]
        hs=[]
        for j in range(len(regions)):
            (x1,y1),(x2,y2)=regions[j]
            x1,y1,x2,y2=correct_rect_ratio(x1,y1,x2,y2)
            x1,y1,x2,y2=add_margin(x1,y1,x2,y2)
            w,h=(abs(x1-x2), abs(y1-y2))
            y1s.append(y1)
            y2s.append(y2)
            h=abs(y1-y2)
            hs.append(h)
        h=int(np.mean(hs))
        w=5*h
        y1=min(y1s)
        y2=max(y2s)
        upper_background = image[0:y1]
        lower_background = image[y2:]
        
        img_h,img_w,_=image.shape
        sampling_x1=0
        sampling_x2=img_w-w
        sampling_upper_y1=0
        sampling_upper_y2=y1-h
        sampling_lower_y1=y2
        sampling_lower_y2=img_h-h
        
        can_sample_upper=sampling_upper_y2>0
        can_sample_lower=sampling_lower_y2>sampling_lower_y1
        
        samples_count=4
        if (can_sample_upper or can_sample_lower) and sampling_x2>0: 
            for i in range(samples_count):
                sampled_x=ra.randint(sampling_x1, sampling_x2)
                sampling_done=False
                if i%2 and can_sample_upper:
                    sampled_y=ra.randint(sampling_upper_y1, sampling_upper_y2)
                    sampling_done=True
                elif can_sample_lower:
                    sampled_y=ra.randint(sampling_lower_y1, sampling_lower_y2)
                    sampling_done=True
                if sampling_done:
                    sample=image[sampled_y:sampled_y+h, sampled_x:sampled_x+w]
                    sample=cv2.resize(sample, (120,24))
                    sample_filename=filename.replace(".jpg", "_"+str(i)+".jpg")
                    backgrounds_path = DS_path+"processed_scenes/backgrounds"
                    os.chdir(backgrounds_path) 
                    try:
                        os.mkdir(scene_name)
                    except FileExistsError:
                        None
                    os.chdir(scene_name) 
                    cv2.imwrite(sample_filename, sample) 
    
def process_image(filename, scene_path, scene_name, label_resolution, labels):
    filepath=scene_path+filename
    regions=labels[filename]
    image = cv2.imread(filepath)
    image = cv2.resize(image, label_resolution)
    border=IMG_SAFETY_BORDER
    image_bordered = cv2.copyMakeBorder(image, border, border, border, border, cv2.BORDER_REFLECT)
    
    extract_pallet_rectangles_from_image(regions, border, image_bordered, filename, scene_name)
    extract_background_rectangles_from_image(regions, image, filename, scene_name)

def process_scene(scene_path, scene_name, label_resolution):
    filenames = sorted(os.listdir(scene_path)) #move all listing to utils as function
    filenames=[f for f in filenames if f != "info.txt"]
    labels=read_labels(scene_name)
    labels={file : regions for file, regions in labels }
    for i in range(len(filenames)):
        filename=filenames[i]
        process_image(filename, scene_path, scene_name, label_resolution, labels)

def process_data(scenes_path):
    scene_dir_names = sorted(os.listdir(scenes_path))
    for i in range(len(scene_dir_names)):
        scene_dir_name=scene_dir_names[i]
        scene_path=scenes_path+scene_dir_name+"/"
        include,_,_,_,label_resolution=read_info(scene_path)
        if include:
            process_scene(scene_path, scene_dir_name, label_resolution)
    print("w_max: ",w_max,"h_max: ",h_max)
    print("w_min: ",w_min,"h_min: ",h_min)

In [ ]:
w_max=h_max=0
w_min=h_min=99999

process_data(scenes_path)